In [16]:
import numpy as np
import pandas as pd
import itertools

In [2]:
from data_preprocessing import *
folder_path = './Prolific/data_all/'
main_survey_df = get_main_survey_data(folder_path)
post_survey_df = get_post_survey_filter_only(folder_path)
main_survey_done_df = main_survey_done_study(main_survey_df,post_survey_df)
mean_wellbeing_robot(main_survey_done_df)
mean_wellbeing_scooter(main_survey_done_df)
df_with_turkid = make_df_participant_based(main_survey_done_df)

/home/zzahedi/project-wellbeing/data_preprocessing.py:235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["1_total"] = df.loc[:,
/home/zzahedi/project-wellbeing/data_preprocessing.py:241: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["3_total"] = df.loc[:,


In [3]:
a1 = df_with_turkid['drive_name_0'].str[3]
ow0 = df_with_turkid['3_total_0']
ow1 = df_with_turkid['1_total_1']
ow2 = df_with_turkid['3_total_1']
oa2 = df_with_turkid['drive_name_1'].str[2]
df = pd.concat([a1, oa2, ow0, ow1, ow2], axis=1)
col = ["a_1", "oa_2", "ow_0", "ow_1", "ow_2"]
df.columns = col
for i in col:
    df[i] = pd.to_numeric(df[i])

# print((df.iloc[0:414,:]['a_0'] == 1).sum(), (df.iloc[0:414,:]['oa_0'] == 1).sum())
# print(df_bn['a_0'], df_bn['oa_0'], df_bn['ow_1'])
# print(df_bn['w_1'], df_bn['ow_1'])

In [4]:
df['diff'] = df['a_1'] - df['oa_2']

# compute the cumulative sum of the diff column
indices = []
for i in range(len(df)):
    num_ones_col1 = df['a_1'].iloc[:i+1].sum()
    num_ones_col2 = df['oa_2'].iloc[:i+1].sum()
    if num_ones_col1 == num_ones_col2:
        indices.append(i)

# find the largest slice where the number of 1's in both columns are equal
max_slice = (0, 0)
for i in range(len(indices)):
    for j in range(i+1, len(indices)):
        start, end = indices[i], indices[j]
        num_ones_col1 = df['a_1'].iloc[start:end+1].sum()
        num_ones_col2 = df['oa_2'].iloc[start:end+1].sum()
        if num_ones_col1 == num_ones_col2 and end-start > max_slice[1]-max_slice[0]:
            max_slice = (start, end)

# print the largest slice where the number of 1's in both columns are equal
print(f"The largest slice where the number of 1's in both columns are equal is from index {max_slice[0]} to {max_slice[1]}.")

The largest slice where the number of 1's in both columns are equal is from index 3 to 244.


In [7]:
print((df.iloc[3:245,:]['a_1'] == 1).sum(), (df.iloc[3:245,:]['oa_2'] == 1).sum())

119 119


In [8]:
indices = []
for i in range(len(df)):
    num_ones_col1 = df['a_1'].iloc[:i+1].sum()
    num_ones_col2 = df['oa_2'].iloc[:i+1].sum()
    if num_ones_col1 == num_ones_col2:
        indices.append(i)

# loop through all possible slices and print the ones where the number of 1's in both columns are equal
slices = []
for i in range(len(indices)):
    for j in range(i+1, len(indices)):
        start, end = indices[i], indices[j]
        num_ones_col1 = df['a_1'].iloc[start:end+1].sum()
        num_ones_col2 = df['oa_2'].iloc[start:end+1].sum()
        if num_ones_col1 == num_ones_col2:
            slices.append((start, end))

# sort the slices by size (largest to smallest)
slices.sort(key=lambda x: x[1]-x[0], reverse=True)

# print the slices
print("Slices where the number of 1's in both columns are equal:")
for slice in slices:
    print(f"From index {slice[0]} to {slice[1]}")

Slices where the number of 1's in both columns are equal:
From index 3 to 244
From index 4 to 244
From index 5 to 244
From index 11 to 244
From index 15 to 244
From index 3 to 213
From index 3 to 212
From index 4 to 213
From index 3 to 211
From index 4 to 212
From index 5 to 213
From index 3 to 210
From index 4 to 211
From index 5 to 212
From index 3 to 209
From index 4 to 210
From index 5 to 211
From index 3 to 208
From index 4 to 209
From index 5 to 210
From index 4 to 208
From index 5 to 209
From index 5 to 208
From index 11 to 213
From index 11 to 212
From index 11 to 211
From index 11 to 210
From index 11 to 209
From index 15 to 213
From index 3 to 200
From index 11 to 208
From index 15 to 212
From index 3 to 199
From index 4 to 200
From index 15 to 211
From index 48 to 244
From index 4 to 199
From index 5 to 200
From index 15 to 210
From index 5 to 199
From index 15 to 209
From index 15 to 208
From index 11 to 200
From index 11 to 199
From index 15 to 200
From index 15 to 199
Fro

In [9]:
df1 = df.iloc[3:245,:]
df1['ror_ow0'] = np.nan
df1['ror_ow1'] = np.nan
df1['ror_ow2'] = np.nan

# extract values where col2 is 1 and col1 is 1, assign to appropriate indices in new column
df1.loc[(df['oa_2'] == 1) & (df1['a_1'] == 1), 'ror_ow0'] = df1.loc[(df1['oa_2'] == 1) & (df1['a_1'] == 1), 'ow_0'].values
df1.loc[(df['oa_2'] == 1) & (df1['a_1'] == 1), 'ror_ow1'] = df1.loc[(df1['oa_2'] == 1) & (df1['a_1'] == 1), 'ow_1'].values
df1.loc[(df['oa_2'] == 1) & (df1['a_1'] == 1), 'ror_ow2'] = df1.loc[(df1['oa_2'] == 1) & (df1['a_1'] == 1), 'ow_2'].values
# extract values where col2 is 1 and col1 is 0, assign to appropriate indices in new column
df1.loc[(df1['oa_2'] == 1) & (df1['a_1'] == 0), 'ror_ow0'] = df1.loc[(df1['oa_2'] == 0) & (df1['a_1'] == 1), 'ow_0'].values
df1.loc[(df1['oa_2'] == 1) & (df1['a_1'] == 0), 'ror_ow1'] = df1.loc[(df1['oa_2'] == 0) & (df1['a_1'] == 1), 'ow_1'].values
df1.loc[(df1['oa_2'] == 1) & (df1['a_1'] == 0), 'ror_ow2'] = df1.loc[(df1['oa_2'] == 0) & (df1['a_1'] == 1), 'ow_2'].values
# extract values where col2 is 0 and col1 is 0, assign to appropriate indices in new column
df1.loc[(df1['oa_2'] == 0) & (df1['a_1'] == 0), 'ror_ow0'] = df1.loc[(df1['oa_2'] == 0) & (df1['a_1'] == 0), 'ow_0'].values
df1.loc[(df1['oa_2'] == 0) & (df1['a_1'] == 0), 'ror_ow1'] = df1.loc[(df1['oa_2'] == 0) & (df1['a_1'] == 0), 'ow_1'].values
df1.loc[(df1['oa_2'] == 0) & (df1['a_1'] == 0), 'ror_ow2'] = df1.loc[(df1['oa_2'] == 0) & (df1['a_1'] == 0), 'ow_2'].values
# extract values where col2 is 0 and col1 is 1, assign to appropriate indices in new column
df1.loc[(df1['oa_2'] == 0) & (df1['a_1'] == 1), 'ror_ow0'] = df1.loc[(df1['oa_2'] == 1) & (df1['a_1'] == 0), 'ow_0'].values
df1.loc[(df1['oa_2'] == 0) & (df1['a_1'] == 1), 'ror_ow1'] = df1.loc[(df1['oa_2'] == 1) & (df1['a_1'] == 0), 'ow_1'].values
df1.loc[(df1['oa_2'] == 0) & (df1['a_1'] == 1), 'ror_ow2'] = df1.loc[(df1['oa_2'] == 1) & (df1['a_1'] == 0), 'ow_2'].values

/tmp/ipykernel_4048933/1993335626.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['ror_ow0'] = np.nan
/tmp/ipykernel_4048933/1993335626.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['ror_ow1'] = np.nan
/tmp/ipykernel_4048933/1993335626.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [18]:
df

,a_1,oa_2,ow_0,ow_1,ow_2,diff
0,0,1,3.428571,4.285714,3.428571,-1
1,0,0,2.714286,2.714286,2.714286,0
2,1,0,5.714286,5.285714,4.857143,1
3,1,1,6.142857,6.285714,5.714286,0
4,1,1,4.142857,5.000000,5.000000,0
...,...,...,...,...,...,...
294,0,1,5.571429,5.857143,5.285714,-1
295,0,1,4.000000,5.571429,6.714286,-1
296,0,0,5.000000,3.571429,4.714286,0
297,1,1,6.714286,6.714286,5.142857,0


In [14]:
df_new = pd.concat([df.iloc[0:3,:], df1, df.iloc[245:301,:]], axis = 0)
df_new.iloc[0:3,:]['ror_ow0'] = df_new.iloc[0:3,:]['ow_0']
df_new.iloc[0:3,:]['ror_ow1'] = df_new.iloc[0:3,:]['ow_1']
df_new.iloc[0:3,:]['ror_ow2'] = df_new.iloc[0:3,:]['ow_2']
df_new.iloc[245:301,:]['ror_ow0'] = df_new.iloc[245:301,:]['ow_0']
df_new.iloc[245:301,:]['ror_ow1'] = df_new.iloc[245:301,:]['ow_1']
df_new.iloc[245:301,:]['ror_ow2'] = df_new.iloc[245:301,:]['ow_2']

/tmp/ipykernel_4048933/338939879.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[0:3,:]['ror_ow0'] = df_new.iloc[0:3,:]['ow_0']
/tmp/ipykernel_4048933/338939879.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[0:3,:]['ror_ow1'] = df_new.iloc[0:3,:]['ow_1']
/tmp/ipykernel_4048933/338939879.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [15]:
df_new

,a_1,oa_2,ow_0,ow_1,ow_2,diff,ror_ow0,ror_ow1,ror_ow2
0,0,1,3.428571,4.285714,3.428571,-1,3.428571,4.285714,3.428571
1,0,0,2.714286,2.714286,2.714286,0,2.714286,2.714286,2.714286
2,1,0,5.714286,5.285714,4.857143,1,5.714286,5.285714,4.857143
3,1,1,6.142857,6.285714,5.714286,0,6.142857,6.285714,5.714286
4,1,1,4.142857,5.000000,5.000000,0,4.142857,5.000000,5.000000
...,...,...,...,...,...,...,...,...,...
294,0,1,5.571429,5.857143,5.285714,-1,5.571429,5.857143,5.285714
295,0,1,4.000000,5.571429,6.714286,-1,4.000000,5.571429,6.714286
296,0,0,5.000000,3.571429,4.714286,0,5.000000,3.571429,4.714286
297,1,1,6.714286,6.714286,5.142857,0,6.714286,6.714286,5.142857


In [20]:
used_indices = []
# loop through the rows and update the values of reordered_ow based on a_0 and reordered_ow
for i in itertools.chain(range(0, 3), range(245, 299)):
    if df_new.iloc[i,:]['a_1'] == 1:
        for j in range(3, 245):
            if df_new.iloc[j,:]['a_1'] == 1 and j not in used_indices:
                df_new.at[i,'ror_ow0']= df_new.iloc[j,:]['ror_ow0']
                df_new.at[i,'ror_ow1']= df_new.iloc[j,:]['ror_ow1']
                df_new.at[i,'ror_ow2']= df_new.iloc[j,:]['ror_ow2']
                used_indices.append(j)
                break
    elif df_new.iloc[i,:]['a_1'] == 0:
        for j in range(3, 245):
            if df_new.iloc[j,:]['a_1'] == 0 and j not in used_indices:
                df_new.at[i,'ror_ow0']= df_new.iloc[j,:]['ror_ow0']
                df_new.at[i,'ror_ow1']= df_new.iloc[j,:]['ror_ow1']
                df_new.at[i,'ror_ow2']= df_new.iloc[j,:]['ror_ow2']
                used_indices.append(j)
                break

In [21]:
df_new

,a_1,oa_2,ow_0,ow_1,ow_2,diff,ror_ow0,ror_ow1,ror_ow2
0,0,1,3.428571,4.285714,3.428571,-1,3.428571,5.000000,4.714286
1,0,0,2.714286,2.714286,2.714286,0,5.571429,2.142857,1.857143
2,1,0,5.714286,5.285714,4.857143,1,6.142857,6.285714,5.714286
3,1,1,6.142857,6.285714,5.714286,0,6.142857,6.285714,5.714286
4,1,1,4.142857,5.000000,5.000000,0,4.142857,5.000000,5.000000
...,...,...,...,...,...,...,...,...,...
294,0,1,5.571429,5.857143,5.285714,-1,5.571429,3.714286,3.571429
295,0,1,4.000000,5.571429,6.714286,-1,3.571429,2.000000,4.000000
296,0,0,5.000000,3.571429,4.714286,0,3.285714,3.285714,3.285714
297,1,1,6.714286,6.714286,5.142857,0,4.714286,5.000000,5.142857


In [23]:
df_new.to_csv('swap_action_3seq.csv')